<a href="https://colab.research.google.com/github/Rogerio-mack/IMT_Banco_de_Dados/blob/main/SQL_SQLite_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Instale o SQLite
Geralmente está pré-instalado no Colab, mas...

```
!pip install sqlite3
```

# 2. Conecte o banco, crie e popule tabelas

In [1]:
!rm mydatabase.db

In [2]:
import sqlite3

# Connect to the SQLite database (create the database file if it doesn't exist)
conn = sqlite3.connect('mydatabase.db')

In [3]:
!ls

mydatabase.db  sample_data


In [4]:
# Create a table
conn.execute('''
    CREATE TABLE IF NOT EXISTS employees (
        id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        department TEXT,
        salary REAL
    )
''')

# Insert data
conn.execute("INSERT INTO employees (name, department, salary) VALUES ('Arthur Del Rio', 'Sales', 50000.0)")
conn.execute("INSERT INTO employees (name, department, salary) VALUES ('Gabriel Lippi', 'Marketing', 60000.0)")
conn.execute("INSERT INTO employees (name, department, salary) VALUES ('Sofia Bueno', 'IT', 75000.0)")
conn.execute("INSERT INTO employees (name, department, salary) VALUES ('Otavio Souza', 'IT', 65000.0)")

# Commit the changes
conn.commit()

# Single Query, *sem cursor*

In [5]:
print("\nSimple query without cursor:")
for row in conn.execute("SELECT * FROM employees WHERE department = 'IT'"):
    print(row)


Simple query without cursor:
(3, 'Sofia Bueno', 'IT', 75000.0)
(4, 'Otavio Souza', 'IT', 65000.0)


In [6]:
temp = conn.execute("SELECT * FROM employees WHERE department = 'IT'")
temp.fetchall()

[(3, 'Sofia Bueno', 'IT', 75000.0), (4, 'Otavio Souza', 'IT', 65000.0)]

# Single Query, *com cursor*

In [7]:
print("\nQuery employing a cursor:")
# Create a cursor object to execute SQL commands
cursor = conn.cursor()

# Query employing a cursor
cursor.execute("SELECT * FROM employees WHERE salary > 55000.0")
results = cursor.fetchall()  # Fetch all results

for row in results:
    print(row)


Query employing a cursor:
(2, 'Gabriel Lippi', 'Marketing', 60000.0)
(3, 'Sofia Bueno', 'IT', 75000.0)
(4, 'Otavio Souza', 'IT', 65000.0)


## Working with data
Por exemplo, empregando o dado no Pandas.

In [8]:
import pandas as pd

df = pd.DataFrame(results)
df.columns = ['id', 'name', 'department', 'salary']
df


,id,name,department,salary
0,2,Gabriel Lippi,Marketing,60000.0
1,3,Sofia Bueno,IT,75000.0
2,4,Otavio Souza,IT,65000.0


# Connection $\times$ Cursor

| Feature              | `conn.execute()`                    | `cursor.execute()`                               |
| -------------------- | ----------------------------------- | ------------------------------------------------ |
| Cursor Management    | Implicit (created and closed)       | Explicit (you create and close)                  |
| Multiple Statements  | Not ideal                           | Suitable for multiple statements using same cursor |
| Result Fetching      | Iteration only                      | `fetchone()`, `fetchall()`, `fetchmany()`        |
| Transactions         | Limited control, `commit()` required for INSERT/UPDATE/DELETE | Full transaction control (`commit()`, `rollback()`)          |
| SQL Injection Protection | Not suitable (vulnerable if variables directly embedded)  | Essential for parameterized queries (prevents SQL injection)   |
| Simplicity           | Simpler for basic queries          | More complex, but more powerful                |


In [9]:
new_department = "IT"
min_salary = 55000
cursor = conn.cursor()
cursor.execute("SELECT name, salary FROM employees WHERE department = ? AND salary > ?", (new_department, min_salary)) # Use Tuple
results = cursor.fetchall()
for row in results:
    print(row)
conn.commit()
cursor.close()

('Sofia Bueno', 75000.0)
('Otavio Souza', 65000.0)


In [10]:
cursor.close()
conn.close()

## Exemplo de Update

In [32]:
conn = sqlite3.connect('mydatabase.db')
cursor = conn.cursor()

employee_name = input("Entre com o nome do funcionário: ")
new_salary = int(input("Entre com o novo salário: "))

try:
    # Execute the UPDATE statement with a parameterized query
    cursor.execute("UPDATE employees SET salary = ? WHERE name = ?", (new_salary, employee_name))

    # Check if any rows were updated
    if cursor.rowcount > 0:
        print(f"Salary updated successfully for {employee_name}.")
    else:
        print(f"No employee found with the name {employee_name}.")

    # Commit the changes
    conn.commit()

except sqlite3.Error as e:
    print(f"An error occurred: {e}")
    conn.rollback()  # Rollback in case of error

finally:
    cursor.close()
    conn.close()

conn = sqlite3.connect('mydatabase.db')

for row in conn.execute("SELECT * FROM employees WHERE name = '" + employee_name + "'"):
    print(row)

Entre com o nome do funcionário: Gabriel Lippi
Entre com o novo salário: 1000000
Salary updated successfully for Gabriel Lippi.
(2, 'Gabriel Lippi', 'Marketing', 1000000.0)


### Atenção:

In [33]:
"SELECT * FROM employees WHERE name = " + employee_name

'SELECT * FROM employees WHERE name = Gabriel Lippi'

In [34]:
"SELECT * FROM employees WHERE name = '" + employee_name + "'"

"SELECT * FROM employees WHERE name = 'Gabriel Lippi'"

# Close connection, boa prática

Libera os recursos do programa (memória, cursores etc.) e do banco de dados (transações pendentes, memória, cursores etc.).

In [35]:
conn = sqlite3.connect('mydatabase.db')

# Close the connection
conn.close()
